In [16]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [52]:
output_dir = "/data/handhygiene/outputs/"
img_dir = os.path.join(output_dir, "images/")
json_dir = os.path.join(output_dir, "jsondata/")
jsondata = os.listdir(json_dir)
jsondata.sort()

In [10]:
#df = pd.read_csv('./handhygiene_label.csv')

In [6]:
#df.to_excel('./handhygiene_label.xlsx')

In [53]:
#### after labeling manually ####
df = pd.read_excel('./handhygiene_label.xlsx')

In [ ]:
!cd /data/openpose && ./build/examples/openpose/openpose.bin --image_dir /data/handhygiene/images/clean/ --write_json /data/handhygiene/outputs/jsondata/ --write_images /data/handhygiene/outputs/images/ --keypoint_scale 3

In [114]:
hh_dict={"img_name":[], "keypoints":[], "category":[], "clip_num":[], "video_id":[], "people_id":[]}

def create_keypoints_csv(jsondata):
    for data in jsondata:
        with open(os.path.join(json_dir, data)) as f:
            dt = json.load(f)
            people = dt.get('people')
            img_name = os.path.splitext(data)[0].strip('_keypoints')

            for row in df.values:
                ## columns: clip_num, date, img_name, video_id, 
                ##          mislabeled, target, not_target0, not_target1, not_target2

                img = row[2]
                mis = row[4]
                target = row[5]
                no_targets = [row[6], row[7], row[8]]
                clip_num = row[0]
                video_id = row[3]

                if img_name != os.path.splitext(img)[0]:
                    continue
                if mis == 1.0: # if mislabeled
                    continue

                for idx, p in enumerate(people):
                    keypoints = p.get('pose_keypoints_2d')
                    keypoints = np.reshape(keypoints, (25, 3))
                    list_coord = keypoints[:,:-1]
                    #list_conf = keypoints[:,-1]
                    list_coord = np.reshape(list_coord, (1, -1))
                    list_coord = list(list_coord)[0]
                    list_strip = ''
                    flist_coord
                    print(list_strip)
                    
                    if idx == target: 
                        hh_dict['img_name'].append(img)
                        hh_dict['keypoints'].append(list_coord)
                        hh_dict['category'].append('clean')
                        hh_dict['clip_num'].append(clip_num)
                        hh_dict['video_id'].append(video_id)
                        hh_dict['people_id'].append(0) # people_id = 0 is always the target

                        # to plot
                        # plt.scatter(list_coord[:,0], list_coord[:,1]) # x, y
                        # plt.xlim(0, 1.0)
                        # plt.ylim(1.0, 0)
                        # plt.show()
                    else:
                        for pidx, nt in enumerate(no_targets):
                            if nt == "NaN":
                                continue
                            if nt == "x":
                                list_coord = np.zeros((1, 50)) # fill zero for missing data
                            if nt == idx:
                                hh_dict['img_name'].append(img)
                                hh_dict['keypoints'].append(list_coord)
                                hh_dict['category'].append('notclean')
                                hh_dict['clip_num'].append(clip_num)
                                hh_dict['video_id'].append(video_id)
                                hh_dict['people_id'].append(pidx+1)
    
            f.close()
    return pd.DataFrame(hh_dict).to_csv('./outputs/feature_keypoints.csv',index=False)

SyntaxError: lambda cannot contain assignment (<ipython-input-114-56123aeb66bf>, line 34)

In [111]:
create_keypoints_csv(jsondata)